# 📊 주식 옵션 거래 시뮬레이터

이 노트북은 삼성전자, SK하이닉스, LG의 실제 주가 데이터를 바탕으로 옵션 거래를 시뮬레이션할 수 있는 도구입니다.

## 시뮬레이터 사용 방법

1. 콜옵션(Call) 또는 풋옵션(Put) 선택
   - 콜옵션: 주가가 오를 것으로 예상할 때
   - 풋옵션: 주가가 내릴 것으로 예상할 때

2. 옵션 계약 조건 설정
   - 만기까지 남은 기간 선택
   - 행사가격 설정
   - 투자 금액 입력

3. 시뮬레이션 실행
   - 실시간 차트 확인
   - 수익률 계산 결과 확인

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# 한국 주식 심볼 설정
symbols = {
    '삼성전자': '005930.KS',
    'SK하이닉스': '000660.KS',
    'LG': '003550.KS'
}

# 데이터 다운로드 (1년치)
end_date = datetime.now()
start_date = end_date - timedelta(days=365)

stock_data = {}
for name, symbol in symbols.items():
    stock_data[name] = yf.download(symbol, start=start_date, end=end_date)
    print(f"{name} 데이터 다운로드 완료")

In [ ]:
# 주식 차트 표시 함수
def display_stock_chart(stock_name):
    data = stock_data[stock_name]
    fig = go.Figure(data=[go.Candlestick(x=data.index,
                                        open=data['Open'],
                                        high=data['High'],
                                        low=data['Low'],
                                        close=data['Close'])])
    
    fig.update_layout(title=f'{stock_name} 주가 차트',
                     xaxis_title='날짜',
                     yaxis_title='가격',
                     height=500)
    
    current_price = data['Close'][-1]
    return fig, current_price

# 옵션 선택 UI 생성
stock_dropdown = widgets.Dropdown(
    options=list(symbols.keys()),
    description='종목 선택:',
    style={'description_width': 'initial'}
)

option_type = widgets.RadioButtons(
    options=[('📈 오를 것 같아요 (콜)', 'call'), ('📉 떨어질 것 같아요 (풋)', 'put')],
    description='옵션 유형:',
    style={'description_width': 'initial'}
)

def on_stock_change(change):
    if change['type'] == 'change' and change['new']:
        fig, current_price = display_stock_chart(change['new'])
        clear_output(wait=True)
        display(stock_dropdown)
        display(option_type)
        fig.show()
        print(f"\n현재 주가: {current_price:,.0f}원")

stock_dropdown.observe(on_stock_change, names='value')

# UI 표시
display(stock_dropdown)
display(option_type)

In [ ]:
# 옵션 계약 설정 UI
expiry_slider = widgets.IntSlider(
    value=30,
    min=1,
    max=90,
    step=1,
    description='만기까지 남은 기간(일):',
    style={'description_width': 'initial'}
)

def get_strike_range(stock_name):
    current_price = stock_data[stock_name]['Close'][-1]
    min_strike = current_price * 0.8
    max_strike = current_price * 1.2
    return min_strike, max_strike

strike_slider = widgets.FloatSlider(
    value=0,
    min=0,
    max=100,
    step=100,
    description='행사가격(원):',
    style={'description_width': 'initial'}
)

investment_input = widgets.IntText(
    value=1000000,
    description='투자금액(원):',
    style={'description_width': 'initial'}
)

def update_strike_range(change):
    if change['type'] == 'change' and change['new']:
        min_strike, max_strike = get_strike_range(change['new'])
        strike_slider.min = min_strike
        strike_slider.max = max_strike
        strike_slider.value = stock_data[change['new']]['Close'][-1]
        strike_slider.step = (max_strike - min_strike) / 100

stock_dropdown.observe(update_strike_range, names='value')

# 계약 설정 UI 표시
display(expiry_slider)
display(strike_slider)
display(investment_input)

In [ ]:
# Black-Scholes 옵션 가격 계산 함수
def black_scholes(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    if option_type.lower() == 'call':
        price = S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    else:  # put option
        price = K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)
    
    return price

# 변동성 계산 함수
def calculate_volatility(stock_data):
    returns = np.log(stock_data['Close']/stock_data['Close'].shift(1))
    return np.std(returns.dropna()) * np.sqrt(252)

from scipy.stats import norm

def calculate_option_price(stock_name, strike_price, days_to_expiry, option_type):
    data = stock_data[stock_name]
    S = data['Close'][-1]  # 현재 주가
    K = strike_price  # 행사가
    T = days_to_expiry / 365  # 만기까지 기간(년)
    r = 0.035  # 무위험 이자율 (한국 국채 금리 가정)
    sigma = calculate_volatility(data)  # 변동성
    
    return black_scholes(S, K, T, r, sigma, option_type)

In [ ]:
# 시뮬레이션 실행 버튼
simulate_button = widgets.Button(
    description='시뮬레이션 시작',
    button_style='success'
)

def run_simulation(b):
    stock_name = stock_dropdown.value
    option_type_selected = option_type.value
    days = expiry_slider.value
    strike = strike_slider.value
    investment = investment_input.value
    
    data = stock_data[stock_name]
    current_price = data['Close'][-1]
    
    # 옵션 가격 계산
    option_price = calculate_option_price(stock_name, strike, days, option_type_selected)
    contracts = investment / (option_price * 100)  # 1계약 = 100주
    
    # 수익률 시뮬레이션
    prices = np.linspace(current_price * 0.5, current_price * 1.5, 100)
    payoffs = []
    
    for price in prices:
        if option_type_selected == 'call':
            payoff = max(0, price - strike) * contracts * 100
        else:
            payoff = max(0, strike - price) * contracts * 100
        payoffs.append(payoff)
    
    # 결과 시각화
    fig = make_subplots(rows=2, cols=1,
                       subplot_titles=(f'현재 주가 대비 수익률 시뮬레이션',
                                     '투자 정보 요약'))
    
    # 수익률 차트
    fig.add_trace(
        go.Scatter(x=prices, y=payoffs, name='수익'),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(x=[current_price, current_price],
                  y=[min(payoffs), max(payoffs)],
                  name='현재 주가',
                  line=dict(color='red', dash='dash')),
        row=1, col=1
    )
    
    # 투자 정보 표시
    fig.add_trace(
        go.Table(
            header=dict(values=['항목', '값'],
                       fill_color='paleturquoise',
                       align='left'),
            cells=dict(values=[
                ['종목', '옵션 유형', '계약 수량', '행사가격', '만기까지 남은 기간', '옵션 가격', '총 투자금액'],
                [stock_name,
                 '콜옵션' if option_type_selected == 'call' else '풋옵션',
                 f'{contracts:.2f}계약',
                 f'{strike:,.0f}원',
                 f'{days}일',
                 f'{option_price:,.0f}원',
                 f'{investment:,.0f}원']
            ]),
            align='left'),
        row=2, col=1
    )
    
    fig.update_layout(height=800, showlegend=True)
    clear_output(wait=True)
    display(stock_dropdown)
    display(option_type)
    display(expiry_slider)
    display(strike_slider)
    display(investment_input)
    display(simulate_button)
    fig.show()

simulate_button.on_click(run_simulation)
display(simulate_button)